In [1]:
import edgar_downloader as ed

In [2]:
ed.download_files_10k('aapl', r'C:\Edgar_EC\test_output', 'gregsmith@kubickgroup.com', min_date='2010-01-01')

In [4]:
import edgar_cleaner as ec

In [5]:
ec.write_clean_html_text_files(r'C:\Edgar_EC\test_output', r'C:\Edgar_EC\test_output_2')

done


In [6]:
import ref_data as ref_data

In [8]:
import edgar_sentiment_wordcount as esw

In [9]:
esw.write_document_sentiments(r'C:\Edgar_EC\test_output_2', r'C:\Edgar_EC\sentiment_factors.csv')

In [11]:
ref_data.get_yahoo_data('2010-01-01', '2023-01-01', 'AAPL')

C:\Edgar_EC\edgar_ec_2\edgar_ec\ref_data.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(ticker_data, sort=False)


,Date,High,Low,Close,volume,1daily_return,2daily_return,3daily_return,5daily_return,10daily_return,Symbol
0,2010-01-04,7.660714,7.585000,7.643214,493729600,0.001729,-0.014205,-0.016027,-0.018223,0.004813,AAPL
1,2010-01-05,7.699643,7.616071,7.656429,601904800,-0.015906,-0.017726,-0.011195,-0.031066,-0.012361,AAPL
2,2010-01-06,7.686786,7.526786,7.534643,552160000,-0.001849,0.004787,-0.004076,-0.001517,-0.013746,AAPL
3,2010-01-07,7.571429,7.466071,7.520714,477131200,0.006648,-0.002232,-0.013582,-0.005461,-0.060927,AAPL
4,2010-01-08,7.571429,7.466429,7.570714,447610800,-0.008821,-0.020096,-0.006274,-0.028540,-0.042032,AAPL
...,...,...,...,...,...,...,...,...,...,...,...
3263,2022-12-19,135.199997,131.320007,132.369995,79592600,-0.000529,0.023268,-0.001058,-0.017678,-0.045403,AAPL
3264,2022-12-20,133.250000,129.889999,132.300003,77432800,0.023809,-0.000529,-0.003326,-0.047317,-0.055027,AAPL
3265,2022-12-21,136.809998,132.750000,135.449997,85928000,-0.023773,-0.026504,-0.040015,-0.043116,-0.043042,AAPL
3266,2022-12-22,134.559998,130.300003,132.229996,77852100,-0.002798,-0.016638,-0.046812,-0.017394,-0.015730,AAPL


In [12]:
import edgar_sentiment_analysis_prep as esap

In [18]:
def full_train_dataset(): # input_min_date = None, input_max_date = None
    '''
    Creates a csv to specified file path
    '''

    tickers_sp100 = ref_data.get_sp100()
    #ed.download_files_10k(‘AAPL’, ‘C:/10k_filings_raw’)
    ed.full_download(tickers_sp100, r'C:\10k_filings_raw', 'gregsmith@kubrickgroup.com', min_date = '2022-01-01', report = '10-K') #min_date = input_min_date, max_date = input_max_date,

    ec.write_clean_html_text_files(r'C:\10k_filings_raw', r'C:\10k_filings_clean')
    
    #df_returns = rf.get_yahoo_data(input_min_date, input_max_date, tickers_sp100, 'daily')
    df_returns = ref_data.get_yahoo_data('2022-01-01', '2023-08-01', tickers_sp100, 'daily') # Need to decide dates as I have to pass a date here
    df_returns.to_csv(r'C:\stock_returns_daily.csv', index=False)
    sentiment_dict = ref_data.get_sentiment_word_dict()

    esw.write_document_sentiments(r'C:\10k_filings_clean', r'C:\sentiment_factors.csv')

    # Load Data
    stock_returns_daily_df = pd.read_csv(r'C:\stock_returns_daily.csv')                         # Load in stock return data
    sentiment_factors_df = pd.read_csv(r'C:\sentiment_factors.csv')                             # Load in sentiment word count data
    #stock_returns_daily_df = pd.read_csv('C:/EDGAR/example_shares_output2.csv')   
    #sentiment_factors_df = pd.read_csv('C:/EDGAR/example_sentiment_analysis.csv')  
    
    # Processing and Feature Engineering
    full_stock_returns_daily_df = esap.stock_returns_prep(stock_returns_daily_df)
    full_sentiment_factors_df = esap.sentiment_factors_prep(sentiment_factors_df)
    
    # Combine Datasets
    full_combined_df = pd.merge(full_stock_returns_daily_df, full_sentiment_factors_df, on = ['Date', 'ticker'], how = 'left')

    full_combined_df.to_csv(r'C:\EDGAR\full_dataset.csv', index = False)              # Will need to decide a better place for this

full_train_dataset()

Exception: Input ticker not recognised